In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv
from pyspark.sql import SparkSession
from azure.storage.blob import BlobServiceClient

In [2]:
def load_enviroment():
    # Enviroment variables loader that works both locally and in DataBricks.
    absolute_path = Path.absolute(Path(os.getcwd()))

    for parent in absolute_path.parents:
        if str(parent).endswith("spark-pipeline"):
            repo_path = parent
            break

    enviroment_variable_path = os.path.join(repo_path, "config", ".env")

    load_dotenv(enviroment_variable_path)

In [3]:
load_enviroment()

KAGGLE_FILE = os.environ["KAGGLE_FILE"]
BRONZE_CONTAINER = os.environ["BRONZE_CONTAINER"]
SILVER_CONTAINER = os.environ["SILVER_CONTAINER"]
AZURE_STORAGE_ACCOUNT = os.environ["AZURE_STORAGE_ACCOUNT"]
AZURE_STORAGE_ACCOUNT_KEY = os.environ["AZURE_STORAGE_ACCOUNT_KEY"]

In [4]:
spark = SparkSession.builder.appName("AzureBlobCSVLoader").getOrCreate()
spark.conf.set(
    f"fs.azure.account.key.{AZURE_STORAGE_ACCOUNT}.blob.core.windows.net",
    AZURE_STORAGE_ACCOUNT_KEY
)

In [ ]:
wasbs_path = f"wasbs://{BRONZE_CONTAINER}@{AZURE_STORAGE_ACCOUNT}.blob.core.windows.net/{KAGGLE_FILE}"
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(wasbs_path)

In [ ]:
df.show()